In [17]:
imagefile= r'C:\Users\roumba\Documents\Software\deep-events\Videos\Video_1\Brightfield\220915_cos7_mitostaygold1_w1Brightfield.ome.tif'
csvfile = r'C:\Users\roumba\Documents\Software\deep-events\Videos\Video_1\Brightfield\labels1b.csv'

from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.io
import plotly.express as px
from scipy import ndimage as ndi
import tensorflow
import imageio
from skimage.segmentation import flood, flood_fill
from scipy import signal
import tensorflow_probability as tfp
import tensorflow as tf

img = Image.open(imagefile)
#tiffarray = np.array(img)

datacsv = pd.read_csv(csvfile)
#data.columns


In [6]:
def get_gaussian(mu, sigma, size):
    mu = ((mu[1]+0.5-0.5*size[1])/(size[1]*0.5), (mu[0]+0.5-0.5*size[0])/(size[0]*0.5))
    sigma = (sigma[0]/size[0], sigma[1]/size[1])
    mvn = tfp.distributions.MultivariateNormalDiag(loc=mu, scale_diag=sigma)
    x,y = tf.cast(tf.linspace(-1,1,size[0]),tf.float32), tf.cast(tf.linspace(-1,1,size[1]), tf.float32)
    # meshgrid as a list of [x,y] coordinates
    coords = tf.reshape(tf.stack(tf.meshgrid(x,y),axis=-1),(-1,2)).numpy()
    gauss = mvn.prob(coords)
    return tf.reshape(gauss, size)


In [15]:
empty_frames= np.zeros((100,img.height,img.width))
empty_frames.shape

(100, 2048, 2048)

In [22]:
empty_frames= np.zeros((100,img.height,img.width))

for row_number in range (0, len(datacsv)):
    framenumber_in_row= int(datacsv.loc[row_number, 'axis-0'])
    fission_ycoord = int(datacsv.loc[row_number, 'axis-1'])
    fission_xcoord = int(datacsv.loc[row_number, 'axis-2'])
    empty_frames[framenumber_in_row,fission_ycoord,fission_xcoord]=1
#imageio.mimwrite('points.tiff', empty_frames)
get_gaussian(empty_frames,0.5,2048)
    

TypeError: 'int' object is not subscriptable